In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1)

In [2]:
# NLTK Stop words

mylist= ['إذ','إذا','إذما','إذن','أف','أقل','أكثر','ألا','إلا','التي','الذي','الذين','اللاتي','اللائي','اللتان','اللتيا','اللتين','اللذان','اللذين','اللواتي','إلى','إليك','إليكم','إليكما','إليكن','أم','أما','أما','إما','أن','إن','إنا','أنا','أنت','أنتم','أنتما','أنتن','إنما','إنه','أنى','أنى','آه','آها','أو','أولاء','أولئك','أوه','آي','أي','أيها','إي','أين','أين','أينما','إيه','بخ','بس','بعد','بعض','بك','بكم','بكم','بكما','بكن','بل','بلى','بما','بماذا','بمن','بنا','به','بها','بهم','بهما','بهن','بي','بين','بيد','تلك','تلكم','تلكما','ته','تي','تين','تينك','ثم','ثمة','حاشا','حبذا','حتى','حيث','حيثما','حين','خلا','دون','ذا','ذات','ذاك','ذان','ذانك','ذلك','ذلكم','ذلكما','ذلكن','ذه','ذو','ذوا','ذواتا','ذواتي','ذي','ذين','ذينك','ريث','سوف','سوى','شتان','عدا','عسى','عل','على','عليك','عليه','عما','عن','عند','غير','فإذا','فإن','فلا','فمن','في','فيم','فيما','فيه','فيها','قد','كأن','كأنما','كأي','كأين','كذا','كذلك','كل','كلا','كلاهما','كلتا','كلما','كليكما','كليهما','كم','كم','كما','كي','كيت','كيف','كيفما','لا','لاسيما','لدى','لست','لستم','لستما','لستن','لسن','لسنا','لعل','لك','لكم','لكما','لكن','لكنما','لكي','لكيلا','لم','لما','لن','وبما','أنهم','إنهم','فبما','لنا','له','لها','لهم','لهما','لهن','لو','لولا','لوما','لي','لئن','ليت','ليس','ليسا','ليست','ليستا','ليسوا','ما','ماذا','متى','مذ','مع','مما','ممن','من','منه','منها','منذ','مه','مهما','نحن','نحو','نعم','ها','هاتان','هاته','هاتي','هاتين','هاك','هاهنا','هذا','هذان','هذه','هذي','هذين','هكذا','هل','هلا','هم','هما','هن','هنا','هناك','هنالك','هو','هؤلاء','هي','هيا','هيت','هيهات','والذي','والذين','وإذ','وإذا','وإن','ولا','ولكن','ولو','وما','ومن','وهو','يا','هي','أفهي','أوهي','فهي','وهي','لهي','هو','أهو','أوهو','أفهو','فهو','وهو','لهو','ولهو','فلهو','هم','أهم','أفهم','أوهم','بهم','فهم','وهم','لهم','ولهم','فلهم','هن','أهن','أوهن','أفهن','فهن','بهن','وهن','لهن','فلهن','ولهن','هما','فهما','لهما','وهما','بهما','ولبهما','فلبهما','أنت','وأنت','لأنت','فأنت','ولأنت','فلأنت','أنتم','وأنتم','لأنتم','فأنتم','فلأنتم','ولأنتم','أنت','وأنتن','لأنتن','فأنتن','فلأنتن','ولأنتن','أنتما','وأنتما','لأنتما','فأنتما','ولأنتما','فلأنتما','نحن','أفنحن','أونحن','أولنحن','ونحن','فنحن','لنحن','ولنحن','فلنحن','أنا','أأنا','أفأنا','أوأنا','وأنا','لأنا','فأنا','ولأنا','فلأنا','هذا','أهذا','أفهذا','أوهذا','بهذا','وهذا','كهذا','لهذا','فهذا','ولهذا','فلهذا','هذه','أهذه','أفهذه','أوهذه','بهذه','وهذه','كهذه','لهذه','فهذه','ولهذه','فلهذه','هذان','أهذان','أفهذان','أوهذان','بهذين','وهذان','كهذين','لهذين','فهذان','ولهذين','فلهذان','هذين','أهذين','أفهذين','أوهذين','بهذين','وهذين','كهذين','لهذان','فهذين','ولهذين','فلهذين','هؤلاء','أهؤلاء','فهؤلاء','وهؤلاء','ولهؤلاء','وبهؤلاء','أفهؤلاء','أوهؤلاء','لهؤلاء','كهؤلاء','بهؤلاء','أكهؤلاء','هاتان','أهاتان','أفهاتان','أوهاتان','وهاتان','كهاتين','لهاتين','فهاتان','ولهاتين','فلهاتان','هاتين','أهاتين','أفهاتين','أوهاتين','بهاتين','وهاتين','كهاتين','لهاتان','فهاتين','ولهاتين','فلهاتين','الذين','اللذان','الذي','التي','اللذين','اللاتي','اللائي','والذين','واللذان','والذي','والتي','واللذين','واللاتي','واللائي','فالذين','فاللذان','فالذي','فالتي','فاللذين','فاللاتي','فاللائي','كالذين','كاللذان','كالذي','كالتي','كاللذين','كاللاتي','كاللائي','للذين','لللذان','للذي','للتي','للذين','للاتي','للائي','بالذين','باللذان','بالذي','بالتي','باللذين','باللاتي','باللائي','أوالذين','أواللذان','أوالذي','أوالتي','أواللذين','أواللاتي','أواللائي','أفالذين','أفاللذان','أفالذي','أفالتي','أفاللذين','أفاللاتي','أفاللائي','وللذين','ولللذان','وللذي','وللتي','وللذين','وللاتي','وللائي','وبالذين','وباللذان','وبالذي','وبالتي','وباللذين','وباللاتي','وباللائي','ثم','فثمة','وثمة','لم','ولم','فلم','سوف','وسوف','فسوف','أفسوف','أوسوف','ولسوف','أن','إن','إنا','أنا','وأن','وإن','وإنا','وأنا','فأن','فإن','فإنا','فأنا','بأن','بإن','بإنا','بأنا','أوأن','أوإن','أوإنا','أوأنا','ولأن','ولإن','ولإنا','ولأنا','لئن','فلئن','ولئن','أولئن','أفلئن','ما','وما','أفما','أوما','ولما','لما','كلما','فلما','عم','عمن','بما','فما','كما','وكما','فكما','قد','وقد','لقد','فقد','لكن','ولكن','فلكن','لكنهم','ولكنهم','فلكنهم','لكنه','ولكنه','فلكنه','لكنها','ولكنها','فلكنها','لكنهما','ولكنهما','فلكنهما','لكنهن','ولكنهن','فلكنهن','إذا','إذ','فإذا','وإذا','فإذ','وإذ','أو','في','وفي','ففي','لفي','وفي','من','ومن','فمن','لمن','أومن','أفمن','كمن','إلى','وإلى','فإلى','أإلى','إلام','علام','حتام','عن','وعن','فعن','أوعن','طالما','لطالما','فطالما','حاشا','وحاشا','وعدا','إلا','فحاشا','فعدا','وإلا','ألا','كلا','نعم','اللهم','كل','فكل','ككل','ولكل','وككل','وكل','لكل','ولكل','فلكل','بكل','وبكل','فبكل','كلهم','فكلهم','ككلهم','ولكلهم','وككل','وكلهم','لكلهم','ولكلهم','فلكلهم','بكلهما','وبكلهما','فبكلهما','كلهن','فكلهن','ككل','ولكل','وككل','وكلكم','لكلكم','ولكلكم','فلكلكم','بكل','وبكل','فبكل','كلهما','وكلهما','فكلهما','فكلهن','كله','وكله','فكله','بكله','لكله','ككله','كلها','وكلها','بكلها','فكلها','لكلها','ككلها','ولكلها','بعض','ببعض','وببعض','فببعض','وبعض','ولبعض','فلبعض','فبعض','كبعض','وكبعض','بعضها','ببعضها','وببعضها','فببعضها','وبعضها','ولبعضها','فلبعضها','فبعضها','كبعضها','وكبعضها','بعضه','ببعضه','وببعضه','فببعضه','وبعضه','ولبعضه','فلبعضه','فبعضه','كبعضه','وكبعضه','بعضهن','ببعضهن','وببعضهن','فببعضهن','وبعضهن','ولبعضهن','فلبعضهن','فبعضهن','كبعضهن','وكبعضهن','بعضهم','ببعضهم','وببعضهم','فببعضهم','وبعضهم','ولبعضهم','فلبعضهم','فبعضهم','كبعضهم','وكبعضهم','بعضهما','ببعضهما','وببعضهما','فببعضهما','وبعضهما','ولبعضهما','فلبعضهما','فبعضهما','كبعضهما','وكبعضهما','جميع','وجميع','فجميع','لجميع','كجميع','جميعهم','وجميعهم','فجميعهم','بجميع','بجميعهم','جميعهن','وجميعهن','ولجميعهم','بجميعهن','كجميعهن','جميعها','وجميعها','فجميعها','لجميعها','كجميعها','جميعه','وجميعه','فجميعه','لجميعه','كجميعه','إلخ','حيث','وحيث','بحيث','ثم','فثم','فثمة','ليس','وليس','لا','ولا','ألا','إلا','كلا','نعم','بلى','أفلا','أولا','قبل','قبله','قبلها','قبلهن','قبلهم','قبلي','قبلك','قبلكم','قبلهما','قبلكما','وقبل','وقبله','وقبلها','وقبلهن','وقبلهم','وقبلي','وقبلك','وقبلكم','وقبلهما','وقبلكما','فقبل','فقبله','فقبلها','فقبلهن','فقبلهم','فقبلي','فقبلك','فقبلكم','فقبلهما','فقبلكما','ولقبلكما','بعد','بعده','بعدها','بعدهن','بعدهم','بعدي','بعدك','بعدكم','بعدهما','بعدكما','وبعد','وبعده','وبعدها','وبعدهن','وبعدهم','وبعدي','وبعدك','وبعدكم','وبعدهما','وبعدكما','فبعد','فبعده','فبعدها','فبعدهن','فبعدهم','ولبعدهم','فبعدي','فبعدك','فبعدكم','فبعدهما','فبعدكما','ولبعدكما','من','منه','منها','منهما','منكما','منهم','منهن','منك','مني','منكن','ومن','ومنه','ومنها','ومنهما','ومنكما','ومنهم','ومنهن','ومنك','ومني','ومنكن','فمن','فمنه','فمنها','فمنهما','فمنكما','فمنهم','فمنهن','فمنك','فمني','فمنكن','أمن','أمنه','أمنها','أمنهما','أمنكما','أمنهم','أمنهن','أمنك','أمني','أمنكن','عن','عنه','عنها','عنهما','عنكما','عنهم','عنهن','عنك','عني','عنكن','وعن','وعنه','وعنها','وعنهما','وعنكما','وعنهم','وعنهن','وعنك','وعني','وعنكن','فعن','فعنه','فعنها','فعنهما','فعنكما','فعنهم','فعنهن','فعنك','فعني','فعنكن','أعن','أعنه','أعنها','أعنهما','أعنكما','أعنهم'
         ,'أعنهن','أعنك','أعني','أعنكن','قد','وقد','فلقد','ولقد','لم','ولم','أولم','فلم','ل','له','لها','لهما','لكما','لهم','لهن','لك','لي','لكن','ول'
         ,'وله','ولها','ولهما','ولكما','ولهم','ولهن','ولك','ولي','ولكن','فل','فله','فلها','فلهما','فلكما','فلهم','فلهن','فلك','فلي','فلكن','أل','أله','ألها'
         ,'ألهما','ألكما','ألهم','ألهن','ألك','ألي','ألكن','في','فيه','فيها','فيهما','فيكما','فيهم','فيهن','فيك','فيي','فيكن','وفي','وفيه','وفيها','وفيهما'
         ,'وفيكما','وفيهم','وفيهن','وفيك','وفيي','وفيكن','ففي','ففيه','ففيها','ففيهما','ففيكما','ففيهم','ففيهن','ففيك','ففيي','ففيكن','أفي','أفيه','أفيها'
         ,'أفيهما','أفيكما','أفيهم','أفيهن','أفيك','أفيي','أفيكن','عند','عنده','عندها','عندهما','عندكما','عندهم','عندهن','عندك','عندي','عندكن','وعند'
         ,'وعنده','وعندها','وعندهما','وعندكما','وعندهم','وعندهن','وعندك','وعندي','وعندكن','فعند','فعنده','فعندها','فعندهما','فعندكما','فعندهم','فعندهن'
         ,'فعندك','فعندي','فعندكن','أعند','أعنده','أعندها','أعندهما','أعندكما','أعندهم','أعندهن','أعندك','أعندي','أعندكن','مع','معه','معها','معهما'
         ,'معكما','معهم','معهن','معك','معي','معكن','ومع','ومعه','ومعها','ومعهما','ومعكما','ومعهم','ومعهن','ومعك','ومعي','ومعكن','فمع','فمعه','فمعها','فمعهما','فمعكما','فمعهم','فمعهن','فمعك','فمعي','فمعكن','أمع','أمعه','أمعها','أمعهما','أمعكما','أمعهم','أمعهن','أمعك','أمعي','أمعكن','إلى','إليه','إليها','إليهما','إليكما','إليهم','إليهن','إليك','إليي','إليكن','وإلي','وإليه','وإليها','وإليهما','وإليكما','وإليهم','وإليهن','وإليك','وإليكن','فإلي','فإليه','فإليها','فإليهما','فإليكما','فإليهم','فإليهن','فإليك','فإليي','فإليكن','أإلي','أإليه','أإليها','أإليهما','أإليكما','أإليهم','أإليهن','أإليك','أإليي','أإليكن','على','عليه','عليها','عليهما','عليكما','عليهم','عليهن','عليك','عليكن','وعلى','وعليه','وعليها','وعليهما','وعليكما','وعليهم','وعليهن','وعليك','وعليي','وعليكن','فعلى','فعليه','فعليها','فعليهما','فعليكما','فعليهم','فعليهن','فعليك','فعليي','فعليكن','أعليه','أعليها','أعليهما','أعليكما','أعليهم','أعليهن','أعليك','أعليكن','إياك','فإياك','وإياك','إياكم','فإياكم','وإياكم','إياكن','فإياكن','وإياكن','إياكما','فإياكما','وإياكما','إياكم','فإياكم','وإياكم','إياهم','فإياهم','وإياهم','إياهن','فإياهن','وإياهن','إياي','فإياي','وإياي','إيانا','فإيانا','وإيانا','كذلك','وكذلك','ولذلك','تلكم','وتلكم','وتلكما','فتلكما','ربما','ولربما','فلربما','وربما','لعل','ولعل','فلعل','ليت','وليت','فليت','يا','أو','لو','لولا','ولو','فلو','فلولا','ولولا','ب','به','بها','بهما','بكما','بهم','بهن','بك','بي','بكن','وب','وبه','وبها','وبهما','وبكما','وبهم','وبهن','وبك','وبي','وبكن','فب','فبه','فبها','فبهما','فبكما','فبهم','فبهن','فبك','فبي','فبكن','أبه','أبها','أبهما','أبكما','أبهم','أبهن','أبك','أبي','أبكن','بين','وبين','فبين','لبين','بينه','وبينه','فبينه','لبينه','بينها','بينهما','وبينهما','فبينهما','ولبينهما','لبينهما','وبينهم','فبينهم','ولبينهم','لبينهم','وبينها','فبينها','ولبينها','لبينها','بينهم','بينهن','لبينهن','وبينهن','فبينهن','بيني','وبيني','فبيني','لبيني','بيننا','فبيننا','وبيننا','لبيننا','خلال','خلالهما','خلالهن','خلاله','خلالها','خلالي','خلالك','خلالنا','وخلال','وخلالهما','وخلالهن','وخلاله','وخلالها','وخلالي','وخلالك','وخلالنا','فخلال','فخلالهما','فخلالهن','فخلاله','فخلالها','فخلالي','فخلالك','فخلالنا','تحت','وتحت','فتحت','ولتحت','تحته','وتحته','فتحته','ولتحته','تحتها','وتحتها','فتحتها','ولتحتها','تحتهم','وتحتهم','فتحتهم','ولتحتهم','تحتهن','وتحتهن','فتحتهن','ولتحتهن','تحتي','وتحتي','فتحتي','ولتحتي','تحتهما','وتحتهما','فتحتهما','ولتحتهما','تحتكم','وتحتكم','فتحتكم','ولتحتكم','تحتكن','وتحتكن','فتحتكن','ولتحتكن','تحتكما','وتحتكما','فتحتكما','ولتحتكما','فوق','وفوق','ففوق','ولفوق','فوقه','وفوقه','ففوقه','ولفوقه','فوقها','وفوقها','ففوقها','ولفوقها','فوقهم','وفوقهم','ففوقهم','ولفوقهم','فوقهن','وفوقهن','ففوقهن','ولفوقهن','فوقي','وفوقي','ففوقي','ولفوقي','فوقهما','وفوقهما','ففوقهما','ولفوقهما','فوقكم','وفوقكم','ففوقكم','ولفوقكم','فوقكن','وفوقكن','ففوقكن','ولفوقكن','فوقكما','وفوقكما','ففوقكما','ولفوقكما','أ','ب','ت','ث','ج','ح','خ','د','ذ','ر','ز','س','ش','ص','ض','ط','ظ','ع','غ','ف','ق','ك','ل','م','ن','ه','و','ي','أي','أيهم','أيها','أيهن','أينا','أيكم','أيهم','أيكن','أيهن','وأي','وأيهم','وأيها','وأيهن','وأينا','وأيكم','وأيهم','وأيكن','وأيهن','فأي','فأيهم','فأيها','فأيهن','فأينا','فأيكم','فأيهم','فأيكن','فأيهن','لأي','لأيهم','لأيها','لأيهن','لأينا','لأيكم','لأيهم','لأيكن','لأيهن','كأي','كأيهم','كأيها','كأيهن','كأينا','كأيكم','كأيهم','كأيكن','كأيهن','ذلك','وذلك','فذلك','أذلك','كذلك','بذلك','لذلك','ذلكم','وذلكم','فذلكم','أذلكم','كمذلكم','بذلكم','لذلكم','ذلكن','وذلكن','فذلكن','أذلكن','كنذلكن','بذلكن','لذلكن','ذلكما','وذلكما','فذلكما','أذلكما','كماذلكما','بذلكما','لذلكما','أن','لأن','لأنهم','لأنهما','لأنكم','لأني','لأنكما','لأنكن','لأني','وأن','ولأن','ولأنهم','ولأنهما','ولأنكم','ولأني','ولأنكما','ولأنكن','ولأني','فأن','فلأن','فلأنهم','فلأنهما','فلأنكم','فلأني','فلأنكما','فلأنكن','فلأني','فلئن','ولئن','فلا','لأنه','بل','مما','ذا','كذا','هل','وهل','فهل','أنه','أنها','أنهم','أنهن','أنهما','أنني','أنكم','أننا','وأنه','وأنها','وأنهم','وأنهن','وأنهما','وأنني','وأنكم','وأننا','فأنه','فأنها','فأنهم','فأنهن','فأنهما','فأنني','فأنكم','فأننا','كأنه','كأنها','كأنهم','كأنهن','كأنهما','كأنني','كأنكم','كأننا','لأنه','لأنها','لأنهم','لأنهن','لأنهما','لأنني','لأنكم','لأننا','أني','وأني','فأني','كأني','لنا','ولنا','فلنا','حتى','وحتى','فحتى','فيما','أما','وأما','إنما','أم','وأما','إما','ثنا','حين','وحين','فحين','يومئذ','ويومئذ','فيومئذ','عندئذ','وعندئذ','فعندئذ','إني','هنا','هناك','وهناك','فهناك','لهناك','عما','وعما','وعم','عم','لما','أنى','أيان'
         ,'ماذا','لماذا','ولماذا','فلماذا','وعلام','وحتام','طالما','قلما','وطالما','فطالما','وقلما','ذاك','ذينك','تينك','ذينك','ليست','وليست','فليست','أليست'
         ,'أوليست','أوليس','فليس','وليس','إنها','بينه','كيف','وكيف','فكيف','لكيف','لن','فلن','ولن','ألن','أنهما','إذن','أن','أوربما','أربما','منذ','كأن'
         ,'فإنك','هكذا','وهكذا','فإنه','فإنها','فإنهم','فإنهن','فإنهما','فإنك','فإنهما','فإنكما','فإنكم','فإني','فإنني','الا','اذا','اذ','الى','فى','آل','الدكتور'
         ,'السعودية','العربية','عبدالله','سعود','خلال','متي','ومتى','متى','فمتى','فأيان','كمن','فيمن','فأما','نا','ثنا','ومما'
         ,'فإننا','فإنا','كان','فكان','وكان','يكون','كانوا','كنا','كنتم','وكنتم','فكنا','وكنا','لكنا','ولكنا','كنت','وكنت','وعندنا','عندنا','اه','أنى','عا'
         ,'انه','انها','كذا','وكذا','فكذا','ال','وهكذا','ؤ','ء','لآ','آ','ا','ئ','ة','لإ','ى','هكذا','لهذا','فلهذا','ولهذا','كانت','إذا','إذما','إذن'
         ,'أف','أقل','أكثر','ألا','إلا','التي','الذي','الذين','اللاتي','اللائي','اللتان','اللتيا','اللتين','اللذان','اللذين','اللواتي','إلى','إليك','إليكم','إليكما'
         ,'إليكن','أم','أما','أما','إما','أن','إن','إنا','أنا','أنت','أنتم','أنتما','أنتن','إنما','إنه','أنى','أنى','آه','آها','أو','أولاء','أولئك','أوه'
         ,'آي','أي','أيها','إي','أين','أين','أينما','إيه','بخ','بس','بعد','بعض','بك','بكم','بكم','بكما','بكن','بل','بلى','بما','بماذا','بمن','بنا','عدد','تم','كورونا','العام'
         ,'به','بها','بهم','بهما','بهن','بي','بين','بيد','تلك','تلكم','تلكما','ته','تي','تين','تينك','ثم','ثمة','حاشا','حبذا','حتى','حيث','حيثما'
         ,'حين','خلا','دون','ذا','ذات','ذاك','ذان','ذانك','ذلك','ذلكم','ذلكما','ذلكن','ذه','ذو','ذوا','ذواتا','ذواتي','ذي','ذين','ذينك','ريث'
         ,'سوف','سوى','شتان','عدا','عسى','عل','على','عليك','عليه','عما','عن','عند','غير','فإذا','فإن','فلا','فمن','في','فيم','فيما','فيه'
         ,'فيها','قد','كأن','كأنما','كأي','كأين','كذا','كذلك','كل','كلا','كلاهما','كلتا','كلما','كليكما','كليهما','كم','كم','كما','كي','كيت','كيف'
         ,'كيفما','لا','لاسيما','لدى','لست','لستم','لستما','لستن','لسن','لسنا','لعل','لك','لكم','لكما','لكن','لكنما','لكي','لكيلا','لم','لما','لن','لنا','له'
         ,'لها','لهم','لهما','لهن','لو','لولا','لوما','لي','لئن','ليت','ليس','ليسا','ليست','ليستا','ليسوا','ما','ماذا','متى','مذ','مع','مما','ممن','من'
         ,'منه','منها','منذ','مه','مهما','نحن','نحو','نعم','ها','هاتان','هاته','هاتي','هاتين','هاك','هاهنا','هذا','هذان','هذه','هذي','هذين','هكذا'
         ,'هل','هلا','هم','هما','هن','هنا','هناك','هنالك','هو','هؤلاء','هي','هيا','هيت','هيهات','والذي','فكانت','وإذ','وإذا','وإن','ولا','ولكن','يمكن','يتم','يجب','حول'
         ,'ولو','وما','ومن','وهو','يا','إذما','إذما','إذما','إذما','والذين','تكون','ان','ولكنتما','وكانتا','فكانتا','كانتا','فكنتم','فكنا','فكن','وكن','كن','وزير','الدكتور','خالد','فيصل','وقال','قال','عبر','فقط'
         ,'يكن','لتكن','ولتكن','فلتكن','ولكانت','لكانت','وان','كونوا','كنتما','عندما','الله','بن']
from nltk.corpus import stopwords
stop_words = mylist
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [9]:
df = pd.read_csv(r'C:\\Users\\HP\\Downloads\\clean_Arabic_Poem Comprehensive_Dataset.csv')
df.drop(columns=to_drop, inplace=True)
del df["a"]
df


,Unnamed: 0,Unnamed: 0.1,العصر,الشاعر,الديوان,القافية,البحر,الشطر الايسر,الشطر الايمن,البيت
0,0,0,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,وَأَن تَجمَعا شَملي وَتَنتَظِرا غَدا,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا وَأَن...
1,1,1,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,وَلا سُرعَتي يَوماً بِسابِقَةِ الرَدى,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ وَلا سُ...
2,2,2,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,وَتَستَوجِبا مَنّاً عَلَيَّ وَتُحمَدا,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً وَتَ...
3,3,3,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,تُؤامِرُني سِرّاً لِأَصرِمَ مَرثَدا,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ تُؤامِرُ...
4,4,4,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,وَأَفرَعَ في لَومي مِراراً وَأَصعَدا,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ وَأَفر...
...,...,...,...,...,...,...,...,...,...,...
1684663,1831765,1831765,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,وأحلى قصيدة تَتَغنى,هي أغلى ما أنشأ اللَّه في الدنيا,هي أغلى ما أنشأ اللَّه في الدنيا وأحلى قصيد...
1684664,1831766,1831766,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,كحلم يغشى الجفون الوسنى,هي أغرودة الأغاريد تنساب,هي أغرودة الأغاريد تنساب كحلم يغشى الجفون ا...
1684665,1831767,1831767,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,يتداعى وجداً ويخفق حسنا,هي شلال بهجة وبهاء,هي شلال بهجة وبهاء يتداعى وجداً ويخفق حسنا
1684666,1831768,1831768,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,يدك الحدود سجناً فسجنا,هي حلم الهوى ومنطلقي الباقي,هي حلم الهوى ومنطلقي الباقي يدك الحدود سجنا...


In [14]:
len(df)


1684668

In [15]:
# Convert to list
data = df.البيت.values.tolist()
#data = df.fullText.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا وَأَن تَجمَعا شَملي وَتَنتَظِرا غَدا']


In [16]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['خليلي', 'لا', 'تستعجلا', 'ان', 'تزودا', 'وان', 'تجمعا', 'شملي', 'وتنتظرا', 'غدا']]


In [17]:


# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['خليلي', 'لا', 'تستعجلا', 'ان', 'تزودا', 'وان', 'تجمعا', 'شملي', 'وتنتظرا', 'غدا']


In [18]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in mylist] for doc in texts]

#def make_bigrams(texts):
   # return [bigram_mod[doc] for doc in texts]

#def make_trigrams(texts):
 #   return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
#data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
#data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized[:1])
print(data_words_nostops[:1])

[['خليلي', 'تستعجلا', 'تزودا', 'تجمعا', 'شملي', 'وتنتظرا', 'غدا']]


In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_nostops)

# Create Corpus
texts = data_words_nostops

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [21]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:4]]

[[('تجمعا', 1),
  ('تزودا', 1),
  ('تستعجلا', 1),
  ('خليلي', 1),
  ('شملي', 1),
  ('غدا', 1),
  ('وتنتظرا', 1)],
 [('الردى', 1),
  ('بسابق', 1),
  ('بسابقة', 1),
  ('سرعتي', 1),
  ('لبث', 1),
  ('مغنم', 1),
  ('يوما', 2)],
 [('اقض', 1),
  ('اليوم', 1),
  ('تنظراني', 1),
  ('علي', 1),
  ('لبانة', 1),
  ('منا', 1),
  ('وتحمدا', 1),
  ('وتستوجبا', 1)],
 [('بجد', 1),
  ('توامرني', 1),
  ('رشيدة', 1),
  ('سرا', 1),
  ('لاصرم', 1),
  ('لعمرك', 1),
  ('مرثدا', 1),
  ('نفس', 1)]]

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=.1,
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
#print("$")
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_nostops, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='tsne')
vis

In [ ]:
import os
os.environ.update({'MALLET_HOME':r'C:\\Users\\HP\\AppData\\Local\\Temp\\mallet-2.0.8\\mallet-2.0.8'})
mallet_path = 'C:\\Users\\HP\\AppData\\Local\\Temp\\mallet-2.0.8\\mallet-2.0.8\\bin\\mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=2, id2word=id2word)

In [ ]:
# Show Topics
#print(ldamallet.show_topics(formatted=True))
print(ldamallet.show_topics(2))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_words_nostops, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

# Compute Coherence Score using UMass
#coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_words_nostops, dictionary=id2word, coherence="u_mass")
#coherence_ldamallet = coherence_model_ldamallet.get_coherence()
#print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=1, step=1):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_words_nostops, start=1,limit=10, step=1)



In [ ]:
import matplotlib 
matplotlib.rc('xtick', labelsize=12) 
matplotlib.rc('ytick', labelsize=12)

font = {'family' : 'sakkal majalla',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)
from arabic_reshaper import arabic_reshaper
from bidi.algorithm import get_display
from matplotlib import pyplot as plt
def f(x):
    out=get_display(arabic_reshaper.reshape(x))
    return(out)
# Show graph
limit=10; start=1; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel(f("عدد الموضوعات"), alpha = 1)
plt.ylabel(f("درجة التماسك"), alpha = 1)
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[6]
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics(num_words=6))

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics